In [138]:
from langchain_ollama import OllamaEmbeddings, ChatOllama
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import TextLoader, UnstructuredExcelLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate, ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationBufferMemory
from operator import itemgetter
from langchain.chains import ConversationChain

In [233]:
import os, sys
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())
OPENAI_API_KEY = os.environ['OPENAI_API_KEY']
print(os.environ['LANGCHAIN_PROJECT'])

llm bootcamp


In [140]:
llm = ChatOllama(model='llama3')
llm.invoke('hola')

AIMessage(content='Hola! ¿Cómo estás?', additional_kwargs={}, response_metadata={'model': 'llama3', 'created_at': '2025-01-07T16:53:44.5058933Z', 'message': {'role': 'assistant', 'content': ''}, 'done_reason': 'stop', 'done': True, 'total_duration': 4508955400, 'load_duration': 2820797700, 'prompt_eval_count': 12, 'prompt_eval_duration': 824117000, 'eval_count': 8, 'eval_duration': 861864000}, id='run-d7b29b11-c81e-4696-8798-773ea95fcaf4-0', usage_metadata={'input_tokens': 12, 'output_tokens': 8, 'total_tokens': 20})

In [234]:
embeddings = OllamaEmbeddings(model='nomic-embed-text')
splitters = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200, add_start_index=True)

In [235]:
sales_loader = UnstructuredExcelLoader('./data/ventas_farmacia_diciembre.xlsx')
excel_docs = sales_loader.load()
text_loader = TextLoader('./data/farmacia.txt')
text_docs = text_loader.load()
pharma_docs = [excel_docs, text_docs]

In [236]:
splitted_docs = []
for d in pharma_docs:
    splitted = splitters.split_documents(d)
    splitted_docs.append(splitted)
docs = [x for sublist in splitted_docs for x  in (sublist if isinstance(sublist, list)else [sublist]) ]
vectorstore = Chroma.from_documents(documents=docs, embedding=embeddings)


In [240]:
retriever = vectorstore.as_retriever(search_type='similarity', search_kwargs={'k':3})

In [219]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [220]:
rag_message = """Eres un asistente de una farmacia. Tu trabajo consiste en proporcionar respuestas a las preguntas que se te hacen teniendo en cuenta los documentos proporcionados.
Importante solo responder a las preguntas que estén relacionadas con el contexto. Si no sabes la respuesta no te la inventes. Utiliza el contexto para responder a la pregunta {question}

contexto: {context}"""
rag_prompt = ChatPromptTemplate.from_template(rag_message)

In [221]:
openai_llm = ChatOpenAI(model='gpt-3.5-turbo')

In [243]:
rag_chain = (
    {'context':  retriever | format_docs, 'question': RunnablePassthrough()}
    | rag_prompt
    | llm
    | StrOutputParser()
)

In [231]:
for chunk in rag_chain.stream('cuantos empleados tiene la farmacia?'):
    print(chunk, end="", flush=True)

La farmacia cuenta con un equipo de 6 empleados.

In [245]:
for chunk in rag_chain.stream('que productos se han vendido en diciembre?'):
    print(chunk, end="", flush=True)

Según el contexto, los productos que se han vendido en diciembre son:

* Ibuprofeno: 15 + 17 + 14 + 4 + 1 + 17 + 19 + 8 = 112 unidades
* Lorazepam: 6 + 12 + 19 + 17 + 10 + 14 + 1 + 13 = 102 unidades
* Minoxidil: 11 + 7 + 3 + 2 + 1 + 15 + 19 + 3 = 73 unidades
* Almax: 17 + 14 + 17 + 3 + 1 + 18 + 10 + 1 = 81 unidades

In [189]:
rag_chain.invoke({'question':'cual es el producto que más se ha vendido en diciembre?', 'history': 'hi'})

'Según el contexto proporcionado, no hay información específica sobre las ventas de productos en diciembre ni sobre el producto que más se vendió ese mes. Por lo tanto, no puedo responder a esa pregunta. Si necesitas saber algo relacionado con los productos o servicios ofrecidos por la farmacia, estoy aquí para ayudarte.'

In [187]:
type(rag_chain)

langchain_core.runnables.base.RunnableSequence

In [199]:
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)
rag_memory_message = """Eres un asistente de una farmacia. Tu trabajo consiste en proporcionar respuestas a las preguntas que se te hacen teniendo en cuenta los documentos proporcionados.
Importante solo responder a las preguntas que estén relacionadas con el contexto. Si no sabes la respuesta no te la inventes. Utiliza el contexto y el historial de la conversación para responder a la pregunta {question}

contexto: {context}
historial:{chat_history}"""
rag_memory_prompt = ChatPromptTemplate.from_template(rag_memory_message)
chat_history = memory.load_memory_variables({})

memory_chain = ({'context': itemgetter('question') | retriever | format_docs, 'question': itemgetter('question'), 'chat_history': itemgetter('history')}
                | rag_memory_prompt
                | llm
                | StrOutputParser()
)
# Datos de entrada
input_data = {
    'question': '¿Cuáles son las funciones de un auxiliar de farmacia?',
    'history': chat_history
}

# Ejecutar la cadena
response = memory_chain.invoke(input_data)
print(response)

# Guardar la nueva interacción en la memoria



Las funciones de un auxiliar de farmacia en Farmacia Miranda Sanz serían brindar asesoramiento y atención en la salud y bienestar de la comunidad, junto a los otros 2 auxiliares y los 3 farmacéuticos que componen el equipo de empleados.


In [204]:
from langchain_core.messages import HumanMessage, AIMessage

In [207]:
HumanMessage(input_data['question'])
inputs = {'question': input_data['question']}
outputs = {'response': response}

In [208]:
memory.save_context(inputs=inputs, outputs=outputs)

In [210]:
chat_history = memory.load_memory_variables({})
chat_history

{'chat_history': [HumanMessage(content='¿Cuáles son las funciones de un auxiliar de farmacia?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Las funciones de un auxiliar de farmacia en Farmacia Miranda Sanz serían brindar asesoramiento y atención en la salud y bienestar de la comunidad, junto a los otros 2 auxiliares y los 3 farmacéuticos que componen el equipo de empleados.', additional_kwargs={}, response_metadata={})]}